Primer paso: "Las importaciones necesarias"  

- sys: se utiliza para modificar el sys.path, permitiendo que Python encuentre módulos personalizados que están fuera del directorio actual del notebook.

- csv: módulo estándar de Python que permite leer y escribir archivos CSV.

- Path de pathlib: para trabajar genericamente con rutas de archivos. Esto es importante porque este proyecto requiere rutas configuradas a través de variables definidas en src/util/constantes.py, para que cumpla con una estructura flexible que se pueda ejecutar en cualquier maquina de manera consistente sin necesidad de escribir rutas absolutas ni relativas manualmente. 

- Se importan dos rutas definidas en constantes.py

- Se importa la función fusionar_csv definida en fusionador.py

In [1]:
import sys
import csv
from pathlib import Path

sys.path.append("..") # Esto es necesario porque el archivo constantes.py están en src/util/, que está fuera del directorio notebooks/.
from src.util.constantes import FILES_PATH, FUSION_PATH
from src.util.fusionador import fusionar_csv

Fusión de los archivos de hogares en un solo archivo

En esta sección se invoca la función fusionar_csv definida en src/util/fusionador.py para buscar y unir todos los archivos usu_hogar_* encontrados en las subcarpetas de files_eph/. El archivo resultante se guarda en la carpeta fusion_eph/ generando el dataset unificado de hogares.

In [2]:
archivo_salida_hogar = FUSION_PATH / "hogares_fusionado.csv"

# Ejecuta la fusión de todos los archivos 'usu_hogar_*'
fusionar_csv("usu_hogar_", FILES_PATH, archivo_salida_hogar)

fusion de archivos generado: hogares_fusionado.csv


Carga de archivo fusionado, guardamos los datos en la variable datos_hogar que usamos para todo el procesamiento que sigue.

In [3]:
with archivo_salida_hogar.open(encoding="utf-8") as f:
    lector = csv.DictReader(f, delimiter=";")
    datos_hogar = list(lector)

Este módulo procesa los datos de los hogares agregando dos nuevas columnas: TIPO_HOGAR y DENSIDAD_HOGAR. La columna TIPO_HOGAR clasifica a cada hogar según la cantidad de personas que lo componen, distinguiendo entre hogares unipersonales, nucleares y extendidos. Por otro lado, DENSIDAD_HOGAR evalúa el nivel de hacinamiento calculando la relación entre la cantidad de personas y el número de habitaciones destinadas al descanso, categorizando el resultado en niveles de baja, media o alta densidad.

In [4]:
from src.util.modulos_seccion_a import punto7_9

punto7_9(datos_hogar)

Se generan una nueva columna llamada MATERIAL_TECHUMBRE que indica el tipo de hogar basado en el campo V4: 
- 5 a 7: "Material precario". 
- 1 a 4: "Material durable". 
- 9: “No aplica”

In [5]:
from src.util.modulos_seccion_a import punto8

punto8(datos_hogar)

Se debe generar una nueva columna llamada CONDICION_DE_HABITABILIDAD, la misma califica a las viviendas y puede tener el valor de: insuficiente, regular, saludables y buena. 
Las reglas que definen la clasificación de las viviendas fueron definidas para que sean estas: 
- Reglas definidas: 
1) Para insuficiente: 
* II4_1 (cuarto de cocina) opción NO
* IV6: (tiene agua) opción fuera del terreno 
* II9: (baño, tenencia y uso) 03 o 04 no tiene baño 
* IV3_ : (piso) opción 3, ladrillo suelto/tierra
* MATERIAL_TECHUMBRE: "Material precario". 

2) Para regular:
* II4_1 (cuarto de cocina) opción SI
* IV6: (tiene agua) opción fuera del terreno 
* II9: (baño, tenencia y uso) opción 02 
* IV3: (piso) opción 2, cemento/ladrillo fijo
* MATERIAL_TECHUMBRE: "Material precario".  

3) Para saludables: 
* II4_1 (cuarto de cocina) opción SI
* IV6: (tiene agua) opción fuera del terreno pero dentro del terreno
* IV8: (tiene baño) opción si opción uso exclusivo del hogar 
* MATERIAL_TECHUMBRE: "Material durable". 
* IV3: (piso) opción 2, cemento/ladrillo fijo, o mosaico / baldosa / madera /cerámica / alfombra. 

4) Para buena: 
* II4_1 (cuarto de cocina) opción SI 
* IV6: (tiene agua) 1. por cañería dentro de la vivienda
* IV8: (tiene baño) opción si opción uso exclusivo del hogar 
* MATERIAL_TECHUMBRE: "Material durable". 
* IV3: (piso) opción 2, cemento/ladrillo fijo, o opción 3 mosaico / baldosa / madera /cerámica / alfombra.

In [6]:
from src.util.modulos_seccion_a import punto10

punto10(datos_hogar)

Para que se guarden los cambios hechos. En el archivo "hogares_fusionado_actualizado.csv" va a mostrar las nuevas columnas en el Excel.

In [7]:
from src.util.modulos_seccion_a import actualizar_archivo

ruta = FUSION_PATH / "hogares_fusionado_actualizado.csv"

actualizar_archivo(datos_hogar, ruta)

In [8]:
from src.util.modulos_seccion_b import aglomerado_mas_viviendas
from src.util.modulos_seccion_b import porcentaje_inquilinos

A partir de hogares_fusionado_actualizado.csv, informa:

El aglomerado con mayor cantidad de viviendas con más de dos ocupantes y sin baño. Informa también la cantidad de ellas.

In [9]:
archivo_actualizado_hogar = FUSION_PATH / "hogares_fusionado_actualizado.csv"
aglomerado_mas_viviendas(archivo_actualizado_hogar)

Aglomerado con más viviendas sin baño y con más de 2 ocupantes: Partidos del GBA
Cantidad de viviendas: 0


Ordena e informa las regiones según el porcentaje de viviendas ocupadas por inquilinos.

In [10]:
porcentaje_inquilinos(archivo_actualizado_hogar)

Regiones ordenadas por porcentaje de viviendas ocupadas por inquilinos de cada Region.
Patagonia: 27.39% de inquilinos.
Pampeana: 25.07% de inquilinos.
Cuyo: 23.56% de inquilinos.
Gran Buenos Aires: 20.53% de inquilinos.
Noreste: 17.57% de inquilinos.
Noroeste: 14.73% de inquilinos.


Se calcula el ranking de los 5 aglomerados con mayor porcentaje de hogares con dos o más personas que completaron estudios universitarios, usando la función ranking_aglomerados_hogares, que pondera los datos con la columna PONDERA.


In [11]:
archivo_indiv = FUSION_PATH / "individuos_fusionado_actualizado.csv"

with archivo_indiv.open(encoding="utf-8") as f:
    lector_indiv = csv.DictReader(f, delimiter=";")
    datos_indiv_act = list(lector_indiv)

archivo_actualizado_hogar = FUSION_PATH / "hogares_fusionado_actualizado.csv"

with archivo_actualizado_hogar.open(encoding="utf-8") as f:
    lector_hog = csv.DictReader(f, delimiter=";")
    datos_hogar_act = list(lector_hog)

In [ ]:
from src.util.modulos_seccion_b import ranking_aglomerados_hogares

ranking = ranking_aglomerados_hogares(datos_hogar_act, datos_indiv_act)

for i, (aglomerado, porcentaje) in enumerate(ranking.items(), start=1):
    print(f"{i}. Aglomerado {aglomerado}: {float(porcentaje):.2f}%")

1. Aglomerado 32: 75.97%
2. Aglomerado 19: 68.10%
3. Aglomerado 23: 64.68%
4. Aglomerado 2: 61.68%
5. Aglomerado 6: 60.73%


Se calcula el porcentaje de viviendas ocupadas por sus propietarios en cada aglomerado usando la función porcentaje_viviendas_propietarios, que pondera los datos con la columna PONDERA. El resultado muestra cómo se distribuye la tenencia de viviendas en cada aglomerado.


In [13]:
from src.util.modulos_seccion_b import porcentaje_viviendas_propietarios

# Calcular el porcentaje de viviendas ocupadas por sus propietarios para cada aglomerado
porcentajes = porcentaje_viviendas_propietarios(datos_hogar_act)

# Informar el resultado
print("Porcentaje de viviendas ocupadas por sus propietarios por aglomerado:")
for aglomerado, porcentaje in porcentajes.items():
    print(f"Aglomerado {aglomerado}: {porcentaje:.2f}%")

Porcentaje de viviendas ocupadas por sus propietarios por aglomerado:
Aglomerado 7: 81.24%
Aglomerado 32: 19.35%
Aglomerado 2: 73.01%
Aglomerado 14: 91.30%
Aglomerado 15: 85.09%
Aglomerado 5: 77.42%
Aglomerado 27: 85.86%
Aglomerado 18: 93.90%
Aglomerado 26: 79.00%
Aglomerado 20: 76.67%
Aglomerado 29: 63.59%
Aglomerado 33: 77.17%
Aglomerado 8: 90.10%
Aglomerado 23: 82.49%
Aglomerado 91: 83.68%
Aglomerado 3: 71.85%
Aglomerado 4: 63.16%
Aglomerado 31: 66.28%
Aglomerado 34: 76.94%
Aglomerado 12: 72.54%
Aglomerado 30: 80.89%
Aglomerado 38: 88.22%
Aglomerado 17: 73.42%
Aglomerado 19: 86.94%
Aglomerado 36: 75.15%
Aglomerado 93: 75.77%
Aglomerado 13: 62.21%
Aglomerado 22: 96.57%
Aglomerado 9: 90.78%
Aglomerado 10: 80.77%
Aglomerado 25: 87.67%
Aglomerado 6: 79.82%


In [ ]:
from src.util.modulos_seccion_b import porcentaje_material

#Solicitamos A un Usuario Que Ingrese un año entre 2020-2025 {Pero solo descargamos 2023 asi que solo funciona 2023}
años= input('Ingrese Un Año Entre 2020-2025')
#invocamos el modulo porcentaje_material que se encuentra en src-util
aux =  porcentaje_material(datos_hogar_act,años)